#### [Raw data](http://www.soest.hawaii.edu/pibhmc/cms/data-by-location/american-samoa/)

- *Observational*: ~6 coral groups TBD; "*_TOAD". 

Note that [Rose](http://www.soest.hawaii.edu/pibhmc/cms/data-by-location/american-samoa/rose-atoll/rose-atoll-habitat/) does not have optical validation data, but does have biological characterizations from prior surveys. This may be useful for predictive comparison only.

Note that Swains does not have any biological layers available.

- *Environmental*: (in progress- TBD based on ongoing literature review); 1-6 from PIBHMC site
    1. Depth (from bty: "*_bty")
    2. Hard vs. soft bottom (from Arc ASCII file: "*_hs")
    3. Rugosity (from Arc ASCII file: "*_rugosity")
    4. Slope (from Arc ASCII file: "*_slope")
    5. Curvature (from bty)
    6. Aspect (from bty)
    7. Buildings (from PacIOOS: "*_bldngs*"; not available for Rose, Swains)
    8. Environmental sensitivity index (from PacIOOS: "AS_all/Env_sens_index")
    9. Village boundaries (from PacIOOS: "AS_all/Villages/")
    10. Distance from shore (from PacIOOS: "AS_all/Shorelines/")
    11. Wetland proximity (from PacIOOS: "AS_all/Wetlands/")
    12. Population 1960-2000 (from PacIOOS: "AS_all/Population/")
    13. ROMS u velocity (to E) (3 km resolution; from PacIOOS: "AS_all/Currents")
    14. ROMS v velocity (to N) (3 km resolution; from PacIOOS: "AS_all/Currents")

Priority locations:  Ofu-Olosega, Ta‘u, Tutuila
Secondary locations: Rose Atoll, Swains Island

All PIBHMC data is available at 5 m resolution for all locations except for Swains Island (5 m for bty, 40 m for hs, rugosity, slope). Using integrated data when available for completeness of coverage.

#### Load libraries

In [8]:
library(raster)
library(rgdal)

rgdal: version: 1.4-4, (SVN revision 833)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.2, released 2017/09/15
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.2, 08 September 2015, [PJ_VERSION: 492]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.3-1 


#### Load raw data

I need to...
- load the observational and predictor data for each region (OfuOlo, Rose, Swains, Tau, Tut) and all regions (AS_all).
- resample rasters to 50 m resolution
- convert TOAD shapefile to 50 m resolution raster, with all 1s prioritized, OR extract raster data to shapefile- still have the issue of overlapping 0s/1s
- exclude bathymetry values < 30 m
- impute missing data as needed

In [14]:
# Ofu-Olosega
setwd('/home/lindsay/samoa_corals_data/')
bldngs <-readOGR(dsn = "OfuOlo/OfuOlo_bldngs/", layer = "bldngs")
plot(bldngs)

OGR data source with driver: ESRI Shapefile 
Source: "/home/lindsay/samoa_corals_data/OfuOlo/OfuOlo_bldngs", layer: "bldngs"
with 33 features
It has 1 fields


Warning message in readOGR(dsn = "OfuOlo/OfuOlo_bldngs/", layer = "bldngs"):
“Z-dimension discarded”

ERROR: Error in stopifnot(is.list(srl)): infinite label point


#### Notes etc.

- Literature review focusing on priority, then secondary, locations; environmental characteristics of the mesophotic in these areas; mesophotic ecology
- [GeoViews walkthrough for visualization](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226)
- Models: standard GLM classifier, random forest classifier, possibly XGBoost or neural network
- [Metrics for evaluation](https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226)